In [1]:
import numpy as np
import pandas as pd

---

In [2]:
events = pd.read_csv("./data/events.csv")
display(events.head())

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


---
Задание 2.1

Сколько записей событий находится в датасете? Ответ введите без пробелов и знаков препинания.

In [3]:
display(events.shape)

(2756101, 5)

---
Задание 2.2

Какие типы событий содержатся в датасете? Выберите все верные варианты.

In [4]:
display(events["event"].value_counts())

event
view           2664312
addtocart        69332
transaction      22457
Name: count, dtype: int64

---
Задание 2.3

Сколько уникальных признаков товара есть в датасете? Ответ вводите без пробелов и знаков препинания.

In [5]:
properties_1 = pd.read_csv("./data/item_properties_part1.csv")
display(properties_1.head())

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [6]:
properties_2 = pd.read_csv("./data/item_properties_part2.csv")
display(properties_2.tail())

,timestamp,itemid,property,value
9275898,1433646000000,236931,929,n12.000
9275899,1440903600000,455746,6,150169 639134
9275900,1439694000000,347565,686,610834
9275901,1433646000000,287231,867,769062
9275902,1442113200000,275768,888,888666 n10800.000 746840 1318567


In [7]:
item_properties = pd.concat([properties_1, properties_2])
item_properties.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [8]:
# Количество уникальных свойств товара
item_properties["property"].nunique()

1104

---

In [9]:
category_tree = pd.read_csv("./data/category_tree.csv")
category_tree.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


> events — датасет с событиями. Колонки:

• timestamp — время события

• visitorid — идентификатор пользователя

• event — тип события

• itemid — идентификатор объекта

• transactionid — идентификатор транзакции, если она проходила


> category_tree   — файл с деревом категорий (можно восстановить дерево).

• category_id — идентификатор категорий

• parent_id — идентификатор родительской категории



> item_properties - файл с свойствами товаров.

• timestamp — момент записи значения свойства

• item_id — идентификатор объекта

• property — свойство, кажется, они все, кроме категории, захешированы

• value — значение свойства


----

#### Приведение данных к нужному формату

In [10]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


In [11]:
category_tree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   categoryid  1669 non-null   int64  
 1   parentid    1644 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 26.2 KB


In [12]:
item_properties.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20275902 entries, 0 to 9275902
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  int64 
 1   itemid     int64 
 2   property   object
 3   value      object
dtypes: int64(2), object(2)
memory usage: 773.5+ MB


In [13]:
events["timestamp"] = pd.to_datetime(events['timestamp'], unit='ms')
item_properties["timestamp"] = pd.to_datetime(item_properties['timestamp'], unit='ms')

In [14]:
display(
    events.head(),
    item_properties.tail()
)

,timestamp,visitorid,event,itemid,transactionid
0,2015-06-02 05:02:12.117,257597,view,355908,NaN
1,2015-06-02 05:50:14.164,992329,view,248676,NaN
2,2015-06-02 05:13:19.827,111016,view,318965,NaN
3,2015-06-02 05:12:35.914,483717,view,253185,NaN
4,2015-06-02 05:02:17.106,951259,view,367447,NaN


,timestamp,itemid,property,value
9275898,2015-06-07 03:00:00,236931,929,n12.000
9275899,2015-08-30 03:00:00,455746,6,150169 639134
9275900,2015-08-16 03:00:00,347565,686,610834
9275901,2015-06-07 03:00:00,287231,867,769062
9275902,2015-09-13 03:00:00,275768,888,888666 n10800.000 746840 1318567


---

Задание 4.1

Какой процент продаж обеспечивают топовые товары (точка отсечения — 1 июля)? Для расчёта используйте топ-3 товаров по числу
транзакций.


In [15]:
# Посмотрим на виды транзакций, нам нужны продажи
events["event"].value_counts()

event
view           2664312
addtocart        69332
transaction      22457
Name: count, dtype: int64

In [16]:
# Разобьем датасет по точке отсечения
top_temp_before = events[
    (events["timestamp"] <= "2015-07-01") & (events["event"] == "transaction")
].reset_index(drop=True)

display(top_temp_before.tail())

,timestamp,visitorid,event,itemid,transactionid
9649,2015-05-31 20:49:54.758,1256020,transaction,75458,3078.0
9650,2015-05-31 20:49:54.727,1256020,transaction,216687,3078.0
9651,2015-06-01 02:11:44.935,530863,transaction,218612,594.0
9652,2015-06-01 04:51:11.264,601615,transaction,141328,3216.0
9653,2015-06-01 04:43:19.527,886166,transaction,234431,8233.0


In [17]:
# Вытащим idшники топ-3 товаров
top_3 = top_temp_before["itemid"].value_counts().head(3).index.to_list()
top_3

[119736, 369447, 7943]

In [18]:
# Возьмем часть датасета для "прогноза"
top_temp_after = events[
    (events["timestamp"] > "2015-07-01") & (events["event"] == "transaction")
].reset_index(drop=True)

display(top_temp_after.tail())


,timestamp,visitorid,event,itemid,transactionid
12798,2015-07-31 21:12:56.570,1050575,transaction,31640,8354.0
12799,2015-07-31 21:57:58.779,861299,transaction,456602,3643.0
12800,2015-07-31 15:48:50.123,855941,transaction,235771,4385.0
12801,2015-07-31 15:12:40.300,548772,transaction,29167,13872.0
12802,2015-07-31 16:09:49.163,1051054,transaction,312728,17579.0


In [19]:
# Возьмем маску по idшникам и расчитаем процент топ-3 в общем числе продаж
naive_rec = top_temp_after[top_temp_after["itemid"].isin(top_3)].shape[0]
naive_rec_sum = top_temp_after.shape[0]

naive_rec_res = naive_rec / naive_rec_sum * 100
round(naive_rec_res, 3)

0.601

---